In [2]:
import os
import os.path as osp
import glob 


WORKING_DIR = '/home/ntphat/projects/medico/medico'
os.chdir(WORKING_DIR)

DATA_DIR = 'data'
coco_annot = osp.join(DATA_DIR, 'coco_annotation.json')

## Inference

In [3]:
import warnings
warnings.simplefilter("ignore", (UserWarning, FutureWarning))

from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import torch
import argparse
import os
import os.path as osp
import numpy as np
import PIL.Image as Image
import cv2
import matplotlib.pyplot as plt

import model.ResUnet.dataset as dataset
from model.ResUnet.core.res_unet import ResUnet
from model.ResUnet.core.res_unet_plus import ResUnetPlusPlus
from model.ResUnet.utils import (
    get_parser,
    get_default_config,
    BCEDiceLoss,
    MetricTracker,
    jaccard_index,
    dice_coeff,
    MyWriter,
    metrics,
)
from model.ResUnet.init_config import setup

def draw_mask(image, mask, save_path=None):
    mask = mask.squeeze(2)
    mask[mask < 0.5] = 0.0
#     y, x = np.nonzero(mask)
    image[np.nonzero(mask)] = (0,255,0)
    if save_path is not None:
        pil_img = Image.fromarray(np.uint8(image)).convert("RGB")
        pil_img.save(save_path)
#         cv2.imwrite(save_path, image)
    return image


checkpoint_dir = osp.join(WORKING_DIR, 'result/resunet++/checkpoints')
cfg = get_default_config()
cfg.merge_from_file('src/model/ResUnet/configs/default_resunet.yaml')

str_cfg = cfg.dump()
print(str_cfg)

CHECKPOINT_PATH: ''
DATA:
  ROOT_DIR: ./data
  TRAIN: train.csv
  TRAIN_IMAGES: images
  TRAIN_MASKS: dfs_masks
  VAL: val.csv
INFERENCE:
  BATCH_SIZE: 10
  MASK_THRES: 0.5
  SAVE_DIR: ./result/resunet++
MODEL:
  IMAGE_SIZE:
  - 512
  - 512
  NAME: res_unet_plus
OUTPUT_DIR: ./result/resunet++
SOLVER:
  BATCH_SIZE: 2
  EARLY_STOPPING: 10
  EPOCH: 100
  LOGGING_STEP: 10
  LR: 0.001
  VALIDATION_EVERY: -1



In [3]:
checkpoint_dir = osp.join(WORKING_DIR, 'result/resunet++/checkpoints')
cfg = get_default_config()
cfg.merge_from_file('src/model/ResUnet/configs/default_resunet.yaml')

batch_size = 2

# Create val dataset:
val_dataset = dataset.ImageDataset(
    cfg, False, transform=transforms.Compose([dataset.ToTensorTarget()])
)
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size, num_workers=4, shuffle=False
)

# Load checkpoint
model = ResUnetPlusPlus(3).cuda()
resume = osp.join(checkpoint_dir, 'best_model.pt')
checkpoint = torch.load(resume)
model.load_state_dict(checkpoint['state_dict'])
print(f"LOADED MODEL SUCCESSFULLY")

from model.ResUnet.utils.visualize import visualize_result
# do inference
model.eval()
with torch.no_grad():
    for idx, data in tqdm(enumerate(val_dataloader)):
        inputs = data['sat_img'].cuda()
        labels = data['map_img'].cuda()
        img_paths = data['image_path']
    
        outputs = model(inputs)
        all_scores = metrics.calculate_all_metrics(outputs, labels)
        img_names = [p.strip().split('/')[-1] for p in img_paths]
        save_paths = [osp.join('.', p) for p in img_names]
        
        imgs = inputs.cpu().permute(0, 2, 3, 1).numpy()
        gts = labels.cpu().permute(0, 2, 3, 1).numpy()
        preds = outputs.cpu().squeeze(0).permute(0, 2, 3, 1).numpy()
        
        print(preds[0].shape)
        visualize_result(imgs, gts, preds, save_paths, all_scores, cfg)
        
#         gt_path = f'src/model/ResUnet/gt.jpg'
#         pred_path = f'src/model/ResUnet/pred.jpg'
#         draw_mask(img, gt, gt_path)
#         draw_mask(img, pred, pred_path)        
        
        break
        

LOADED MODEL SUCCESSFULLY


0it [00:00, ?it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(512, 512, 1)
[[<AxesSubplot:> <AxesSubplot:>]]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


[[<AxesSubplot:> <AxesSubplot:>]]


0it [00:00, ?it/s]


<Figure size 512x512 with 0 Axes>

<Figure size 512x512 with 0 Axes>

In [ ]:
a = torch.tensor(1.0)
b = max(a, torch.tensor(2.0))
print(b)

TypeError: can't multiply sequence by non-int of type 'float'